# Perform bottom line test for SMP-16-010

We will follow stat comm recommendations here: 

https://twiki.cern.ch/twiki/bin/viewauth/CMS/ScrecUnfolding

If we define $r$ to be the reconstructed MC, $R$ to be the response matrix, $g$ to be the generator-level truth, $u$ to be the unfolded reconstructed MC, $\sigma$ to be the uncertainties returned by the unfolding, $cov$ to be the covariance matrix of the inputs, and $|\cdot|^2$ denotes the L1 norm, we compute:


- $\chi^2_{smeared} = |(r - Rg)/\sqrt{cov}|^2$
- $\chi^2_{unfolded} = |(u - g)/\sigma|^2$

## Set up the response matrices and vectors

In [1]:
import ROOT
ROOT.gSystem.Load("RooUnfold/libRooUnfold")
import math

from ROOT import gRandom, TH1, cout, TH2, TLegend, TFile
from ROOT import RooUnfoldResponse
from ROOT import RooUnfold
from ROOT import RooUnfoldBayes
from ROOT import TCanvas

from HistDriver import *
ROOT.gStyle.SetPadRightMargin(0.2)
ROOT.gStyle.SetOptStat(000000)

histDriver = HistDriver()

extension = ''

mcfile = TFile('responses_rejec_tightgen_otherway_qcdmc_2dplots.root')
datafile = TFile('jetht_weighted_dataplots_otherway_rejec.root')

response = mcfile.Get('2d_response'+ extension)
responseSD = mcfile.Get('2d_response_softdrop' + extension)

truth = mcfile.Get('PFJet_pt_m_AK8Gen')
truthSD = mcfile.Get('PFJet_pt_m_AK8SDgen')
reco = mcfile.Get('PFJet_pt_m_AK8')
recoSD = mcfile.Get('PFJet_pt_m_AK8SD')

truth.SetTitle("Generator mass vs p_{T};Mass (GeV);p_{T} (GeV)")

unfold = RooUnfoldBayes(response, reco, 4)    
unfoldSD = RooUnfoldBayes(responseSD, recoSD, 4)



Welcome to JupyROOT 6.10/08


## Perform the unfolding, get the various matrices and vectors

In [2]:

reco_unfolded = unfold.Vreco()
reco_det = unfold.Vmeasured()
response = unfold.response()
mresponse = response.Mresponse()
truthV = response.Vtruth()
truth_foldedV = ROOT.TVectorD(truthV)
truth_foldedV *= mresponse
errs = unfold.Ereco()
errsV = unfold.ErecoV(2)

reco_unfoldedSD = unfoldSD.Vreco()
reco_detSD = unfoldSD.Vmeasured()
responseSD = unfoldSD.response()
mresponseSD = responseSD.Mresponse()
truthSDV = responseSD.Vtruth()
truth_foldedSDV = ROOT.TVectorD(truthSDV)
truth_foldedSDV *= mresponseSD
errsSD = unfoldSD.Ereco()
errsVSD = unfoldSD.ErecoV(2)

Add truth bin for 382.997 fakes
Now unfolding...
Iteration : 0
Chi^2 of change 4.09551
Iteration : 1
Chi^2 of change 0.801359
Iteration : 2
Chi^2 of change 0.452371
Iteration : 3
Chi^2 of change 0.296706
Calculating covariances due to number of measured events
Add truth bin for 381.188 fakes
Now unfolding...
Iteration : 0
Chi^2 of change 2.86135
Iteration : 1
Chi^2 of change 0.594219
Iteration : 2
Chi^2 of change 0.209456
Iteration : 3
Chi^2 of change 0.0903616
Calculating covariances due to number of measured events


## Compute $\chi^2$ values for ungroomed case:

In [4]:

chi2_1 = 0.0



for ibin in xrange( reco_unfolded.GetNrows() ):
    #print reco_unfolded[ibin], ', ', truthV [ibin], ', ', errsV[ibin]
    if errsV[ibin] > 0.0:
        chi2_1 += pow( (reco_unfolded[ibin] - truthV[ibin]) / errsV[ibin], 2.0)
    
chi2_1 /= reco_unfolded.GetNrows()
print ' Unfolded chi2 : ', chi2_1


chi2_2 = 0.0
cov = unfold.GetMeasuredCov()
for ibin in xrange( reco_det.GetNrows() ):
    #print reco_det[ibin], ', ', truth_foldedV [ibin], ', ', math.sqrt(cov[ibin][ibin])
    if cov[ibin][ibin] > 0.0:
        chi2_2 += pow( (reco_det[ibin] - truth_foldedV[ibin]) / math.sqrt(cov[ibin][ibin]), 2.0)
    
chi2_2 /= reco_det.GetNrows()
print ' Folded   chi2 : ', chi2_2



 Unfolded chi2 :  6.72144967081
 Folded   chi2 :  9.73778052955


## Compute $\chi^2$ values for groomed case:

In [5]:

chi2_1sd = 0.0



for ibin in xrange( reco_unfoldedSD.GetNrows() ):
    #print reco_unfolded[ibin], ', ', truthV [ibin], ', ', errsV[ibin]
    if errsVSD[ibin] > 0.0:
        chi2_1sd += pow( (reco_unfoldedSD[ibin] - truthSDV[ibin]) / errsVSD[ibin], 2.0)
    
chi2_1sd /= reco_unfoldedSD.GetNrows()
print ' Unfolded chi2 : ', chi2_1sd


chi2_2sd = 0.0
covSD = unfoldSD.GetMeasuredCov()
for ibin in xrange( reco_detSD.GetNrows() ):
    #print reco_detSD[ibin], ', ', truth_foldedSDV [ibin], ', ', math.sqrt(covSD[ibin][ibin])
    if covSD[ibin][ibin] > 0.0:
        chi2_2sd += pow( (reco_detSD[ibin] - truth_foldedSDV[ibin]) / math.sqrt(covSD[ibin][ibin]), 2.0)
    
chi2_2sd /= reco_detSD.GetNrows()
print ' Folded   chi2 : ', chi2_2sd




 Unfolded chi2 :  1.79628135232
 Folded   chi2 :  6.73571327896
